In [ ]:
import numpy as np

from models import ConsensusModel, ConsensusLogisticModel
from methods import TrackingADMM, DPMM, APAPC, Main, EXTRA
from utils import save_object, load_object

from matplotlib import pyplot as plt

myparams = {
    #'text.usetex': True,
    #'text.latex.preamble': r'\usepackage{amsfonts}',
    #'font.family': 'Djvu Serif',
    'font.size': 16,
    'axes.grid': True,
    'grid.alpha': 0.3,
    'lines.linewidth': 2
}

plt.rcParams.update(myparams)

%config InlineBackend.figure_format = "retina"
%load_ext autoreload
%autoreload 2

In [ ]:
np.random.seed(307)

n = 5
d = 10
graph = 'ring'
# graph = 'erdos-renyi'
# average_degree = 2

model = ConsensusModel(n, d, graph, edge_prob=average_degree/n, gossip=False, condition_number=10000)
# model = ConsensusLogisticModel(n, d, graph, edge_prob=average_degree/n, gossip=False)

np.linalg.svd(model.C[3], compute_uv=False)

In [ ]:
# titles = ['Main', 'EXTRA']
# results = {}
# #results = {title: {} for title in titles}
num_steps = 10000

output = Main(num_steps=num_steps, model=model)
results['Main'] = output

output = EXTRA(num_steps=num_steps * 30, model=model)
results['EXTRA'] = output

In [ ]:
filename = f'{graph}_n-{model.n}_d-{model.d}_m-{model.m}'
filename_fig = 'figs/example/' + filename + '.pdf'
filename_dump = 'dumps/example/' + filename + '.pkl'

#results = load_object(filename_dump)

fig, ax = plt.subplots(1, 2, figsize=(10, 4), sharey=True)

for title in results.keys():
    output = results[title]
    x_vals = np.cumsum(output['grad_calls'])
    y_vals = output['x_err']
    if title == 'Main':
        title = 'Main algorithm'
    ax[0].plot(x_vals, y_vals, label=title)
ax[0].legend(loc='upper right')
ax[0].yaxis.set_tick_params(labelleft=True)
# ax[0].set_xlim(left=0, right=10_000)
# ax[0].set_ylim(bottom=1e-21, top=1e3)
ax[0].set_yscale('log')
ax[0].set_xlabel("Gradient calls")
ax[0].ticklabel_format(axis='x', style='sci', scilimits=(0, 0))

# for title in results.keys():
#     output = results[title]
#     x_vals = np.cumsum(output['mults_A'])
#     y_vals = output['x_err']
#     if title == 'Main':
#         title = 'Main algorithm'
#     ax[1].plot(x_vals, y_vals, label=title)
# ax[1].legend(loc='upper right')
# ax[1].yaxis.set_tick_params(labelleft=True)
# # ax[1].set_xlim(left=0, right=400_000)
# ax[1].set_yscale('log')
# ax[1].set_xlabel("Mult. by $\mathbf{A}$ and $\mathbf{A}^T$")
# ax[1].ticklabel_format(axis='x', style='sci', scilimits=(0, 0))

for title in results.keys():
    output = results[title]
    x_vals = np.cumsum(output['communications'])
    y_vals = output['x_err']
    if title == 'Main':
        title = 'Main algorithm'
    ax[1].plot(x_vals, y_vals, label=title)
ax[1].legend(loc='upper right')
ax[1].yaxis.set_tick_params(labelleft=True)
# ax[1].set_xlim(left=0, right=400_000)
ax[1].set_yscale('log')
ax[1].set_xlabel("Communications")
ax[1].ticklabel_format(axis='x', style='sci', scilimits=(0, 0))

#plt.legend(bbox_to_anchor=(1, 0.5), loc="center left")
fig.text(0, 0.5, r"$\| x^k - x^* \|_2^2$", va='center', rotation='vertical')
graph_name = 'Ring' if graph == 'ring' else 'Erdős–Rényi'
#plt.suptitle(f'{graph_name}, $n = {model.n}$, $d_i = {model.d}$, $m = {model.m}$', fontsize=24)
plt.tight_layout()
# plt.savefig(filename_fig, bbox_inches='tight')
# save_object(results, filename_dump)
plt.show()